In [22]:
def split_path_name(path):
    path = str(path).split('\\')[-1]

    return path

In [23]:
def transform_labels(labels_list):
    
    unique_values = list(set(labels_list))
    
    for idx, v in enumerate(unique_values):
        labels_list = list(map(lambda x: idx if v == x else x ,  labels_list))
        
    return labels_list

In [24]:
from pathlib import Path
import tensorflow as tf
import skimage as ski
import numpy as np
from skimage.color import gray2rgb


keras = tf.keras 


def import_images(paths:str ='dataset' , target_size:(int,int) = (64,64) ) -> [dict,dict,dict] :
    paths = Path(paths)

    images = {
        'folders': {},
        'input_values' : {}
    }
    
    

    for class_path in Path(paths).iterdir():

        class_path_splitted = split_path_name(class_path)

        images['folders'][class_path_splitted] = {}
        
        
        images['input_values'][class_path_splitted] = {
            'imgs':[],
            'labels':[]
        }

        for dir_name in Path(class_path).iterdir():

            dir_name_splitted = split_path_name(dir_name)

            images['folders'][class_path_splitted][dir_name_splitted] = []
            
            
            for file_name in Path(dir_name).iterdir():
                try :
                    
                    print(file_name)
                    file_extension = str(file_name).split('.')[-1]
                    if file_extension != 'jpeg' and file_extension != 'jpg':
                        raise Exception( f'You can not have forbidden file  extensions in dataset, this file is incorrect : {file_name}')
    
                    
                    img = ski.io.imread(str(file_name))
                    
                    if len(img.shape) == 2:
                          img = gray2rgb(img)
                        
                    elif len(img.shape) > 3:
                        raise Exception(f'This image is not correct, it has more than 3 dimensions: {file_name}')
                    
                    image_resized = ski.transform.resize(img, target_size)  
                    img_as_float = ski.util.img_as_float(image_resized)
                    
                except Exception as e:
                    print(e)
                    return
                
                
              
                
           
                images['folders'][class_path_splitted][dir_name_splitted].append(img_as_float)
                
             
                images['input_values'][class_path_splitted]['imgs'].append(img_as_float)
                
                images['input_values'][class_path_splitted]['labels'].append(dir_name_splitted)
                
        
        images['input_values'][class_path_splitted]['imgs'] =  np.array(images['input_values'][class_path_splitted]['imgs'])
        
        
        one_hot_encoded = keras.utils.to_categorical(transform_labels(   images['input_values'][class_path_splitted]['labels']) , num_classes = 2 )
       
        images['input_values'][class_path_splitted]['labels'] =    one_hot_encoded
                   
    
    return  images['input_values'] , images['folders'],  images




In [25]:


input_values , all_folders, all_imgs  = import_images()

dataset\test\empty_class\empty_classroom_100.jpeg
dataset\test\empty_class\empty_classroom_2.jpeg
dataset\test\empty_class\empty_classroom_20.jpeg
dataset\test\empty_class\empty_classroom_23.jpeg
dataset\test\empty_class\empty_classroom_33.jpeg
dataset\test\empty_class\empty_classroom_36.jpeg
dataset\test\empty_class\empty_classroom_4.jpeg
dataset\test\empty_class\empty_classroom_70.jpeg
dataset\test\empty_class\empty_classroom_81.jpeg
dataset\test\empty_class\empty_classroom_86.jpeg
dataset\test\empty_class\empty_classroom_93.jpeg
dataset\test\full_class\full_classroom_100.jpeg
dataset\test\full_class\full_classroom_19.jpeg
dataset\test\full_class\full_classroom_2.jpeg
dataset\test\full_class\full_classroom_22.jpeg
dataset\test\full_class\full_classroom_32.jpeg
dataset\test\full_class\full_classroom_35.jpeg
dataset\test\full_class\full_classroom_39.jpeg
dataset\test\full_class\full_classroom_7.jpeg
dataset\test\full_class\full_classroom_80.jpeg
dataset\test\full_class\full_classroom_8

In [26]:
X_train,y_train = input_values['train']['imgs'], input_values['train']['labels']


In [27]:
X_train.shape, y_train.shape

((141, 64, 64, 3), (141, 2))

In [28]:
X_val,y_val = input_values['val']['imgs'], input_values['val']['labels']

In [29]:
X_test,y_test = input_values['test']['imgs'], input_values['test']['labels']

In [30]:
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_val_flat = X_val.reshape(X_val.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)



In [31]:
X_train_flat.shape, y_train.shape, X_val_flat.shape, y_val.shape, X_test_flat.shape, y_test.shape

((141, 12288), (141, 2), (40, 12288), (40, 2), (22, 12288), (22, 2))

In [32]:
# Task 1
def initializer(nn_architecture):
    '''
    Given
    - the number of layers 
    - the number of units in each layer
    initializes network parameters (weights and biases)
    
    Arguments
        nn_architecture: a list of integers
                         length of the list is the number of layers
                         each item in the list specifies 
                         the number of units in that layer
    Returns
        weights: list of numpy arrays, w0, w1, ...
        biases: list of numpy arrays, b0, b1, ...
        
    '''
    weights = []
    biases = []
    
    for idx, layer in enumerate(nn_architecture):
        input_shape = layer
        
        if idx + 1 < len(nn_architecture):
            output_shape = nn_architecture[idx +1 ]
            weights.append(np.random.randn(input_shape , output_shape))
            biases.append(np.random.randn(1 , output_shape ))
           
    return weights, biases

In [33]:
input_dim = X_train_flat.shape[1]

In [34]:
input_dim

12288

In [35]:
weights, biases = initializer([input_dim, 30, 2])

In [36]:
weights[0].shape, biases[0].shape

((12288, 30), (1, 30))